In [3]:
import glob
import os
from config_utils import *
import copy

exp_configs = load_configs_from_yaml("ETT_exp_test.json")

folder_path = "./dataset/ETT-small/"  # Replace with the actual path
list_of_configs = []
# Get a list of all CSV files in the specified folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

prediction_lengths = [96, 192, 336, 720]

for config in exp_configs:
    for file_path in csv_files: # ABLATION FOR DATASETS
        config_copy = copy.deepcopy(config)
        # Get the full path
        full_relative_path = file_path
        
        # Get the filename (including extension)
        file_name = os.path.basename(file_path)
        
        print(f"Full Path: {full_relative_path}")
        print(f"File Name: {file_name}")
        print("-" * 20)
        config_copy.data.root_path = full_relative_path
        config_copy.data.data_path = file_name

        for prediction_length in prediction_lengths:
            config_copy_nested = copy.deepcopy(config_copy)
            config_copy_nested.forecast.pred_len = prediction_length
            
            list_of_configs += [config_copy_nested]

save_configs_to_json(list_of_configs, output_file="generated_exp_configs.json")

Full Path: ./dataset/ETT-small/ETTh1_w1501_pfarm.csv
File Name: ETTh1_w1501_pfarm.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w751_pmutual_info.csv
File Name: ETTh1_w751_pmutual_info.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w1251_iprollcorr.csv
File Name: ETTh1_w1251_iprollcorr.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w1501_ipmutual_info.csv
File Name: ETTh1_w1501_ipmutual_info.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w751_pentropy.csv
File Name: ETTh1_w751_pentropy.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_zeroed.csv
File Name: ETTh1_zeroed.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w751_iprollcorr.csv
File Name: ETTh1_w751_iprollcorr.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w751_prollcorr.csv
File Name: ETTh1_w751_prollcorr.csv
--------------------
Full Path: ./dataset/ETT-small/ETTh1_w1251_ipentropy.csv
File Name: ETTh1_w1251_ipentropy.csv
----